In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import scipy as sp
import sys
sys.path.append('../corv/src')
sys.path.append('../wdtools')

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm
import pandas as pd
import corv

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c
import data_selector as ds
import photoradius as pr

import wdtools
import corv


using local paths
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.
Loading pixel_info ...
Loading samples ...


In [ ]:
d1 = pd.read_csv('data/wd_spectra.csv')
d1 = Table.from_pandas(d1)

catalog = Table.read('data/00_raw.fits')
print(len(catalog))

In [ ]:
spAll = Table.read('~/Downloads/spAll-master.fits')

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join_skycoord
from astropy import table

spAll = spAll[spAll['DECCAT'] < 90]
spAll = spAll[spAll['DECCAT'] > -90]

catalog['wd_pos'] = SkyCoord(catalog['wd_ra'], catalog['wd_dec'], unit='deg')
spAll['wd_pos'] = SkyCoord(spAll['RACAT'], spAll['DECCAT'], unit='deg')

join_func = table.join_skycoord(5 * u.arcsecond)
sdss5_catalog = table.join(catalog, spAll, join_funcs={'wd_pos': join_skycoord(5 * u.arcsec)})

In [ ]:
print(sdss5_catalog[['wd_pos_id', 'wd_pos_1', 'wd_pos_2']])

In [ ]:
from astropy.table import unique, Table

def flux_to_mag(flux):
    return 22.5 - 2.5 * np.log10(flux)

#sdss5_catalog = sdss5_catalog[sdss5_catalog['CLASS'] == b"STAR"]

keep = []

for i in range(len(sdss5_catalog['CLASS'])):
    if (sdss5_catalog['CLASS'][i] == sdss5_catalog['CLASS'][-1]) and ('WD' in sdss5_catalog['SUBCLASS'][i]):
        keep.append(1)
    else:
        keep.append(0)
        
sdss5_catalog['KEEP'] = keep        
sdss5_catalog = sdss5_catalog[sdss5_catalog['KEEP'] == 1]

sdss5_catalog['UGRIZ'] = (flux_to_mag(sdss5_catalog['SPECTROFLUX']))
sdss5_catalog['e_UGRIZ'] = (1 / sdss5_catalog['SPECTROFLUX_IVAR']) / sdss5_catalog['SPECTROFLUX']

sdss5_catalog[['wd_pos_id','SPECOBJID','wd_l', 'wd_b', 'ms_parallax','UGRIZ', 'e_UGRIZ']].pprint(max_lines=-1, max_width=-1)
sdss5_catalog = unique(sdss5_catalog, keys=['wd_pos_id'])
sdss5_catalog[['wd_pos_id','SPECOBJID','wd_l', 'wd_b','UGRIZ', 'ms_parallax', 'e_UGRIZ']].pprint(max_lines=-1, max_width=-1)

In [ ]:
result = pr.find_radius(sdss5_catalog['wd_l'][0:1], sdss5_catalog['wd_b'][0:1], sdss5_catalog['ms_parallax'][0:1], sdss5_catalog['UGRIZ'][0:1], sdss5_catalog['e_UGRIZ'][0:1], make_plot = True)